In [2]:
import torch
import sys
import numpy as np
sys.path.append('/home/cyang314/ucr_work/add-thin')

from add_thin.metrics import MMD, lengths_distribution_wasserstein_distance
from add_thin.evaluate_utils import get_task, get_run_data

In [3]:
# Set run id and paths
RUN_ID = "naflz4ti"

WANDB_DIR = "/home/cyang314/ucr_work/add-thin/wandb"
PROJECT_ROOT = "/home/cyang314/ucr_work/add-thin/"  # should include data folder

In [ ]:
def sample_model(task, tmax, n=4000):
    """
    Unconditionally draw n event sequences from Add Thin.
    """
    with torch.no_grad():
        samples = task.model.sample(n, tmax=tmax.to(task.device)).to_time_list()

    assert len(samples) == n, "not enough samples"
    x_N = task.model.get_x_N()
    assert x_N != None
    # assert len(x_N) == n, "not enough samples"
    return samples, x_N

223.54166666666666
198.16666666666669
70.28472222222223


In [ ]:
# Get run data
data_name, seed, run_path = get_run_data(RUN_ID, WANDB_DIR)

# Get task and datamodule
task, datamodule = get_task(run_path, density=True, data_root=PROJECT_ROOT)

# Get test sequences
test_sequences = []
for (
    batch
) in (
    datamodule.test_dataloader()
):  # batch is set to full test set, but better be safe
    test_sequences = test_sequences + batch.to_time_list()

# Sample event sequences from trained model
samples, x_N = sample_model(task, datamodule.tmax, n=4000)
stored_x_0_list = []
stored_x_N_list = []
stored_pair = []
for k in range(1000):
    store_samples, store_x_N = sample_model(task, datamodule.tmax, n=4000)
    stored_pair.append([store_samples, store_x_N])

np.save('./stored_pair.npy', stored_pair)

# Evaluate metrics against test dataset
mmd = MMD(
    samples,
    test_sequences,
    datamodule.tmax.detach().cpu().item(),
)[0]
wasserstein = lengths_distribution_wasserstein_distance(
    samples,
    test_sequences,
    datamodule.tmax.detach().cpu().item(),
    datamodule.n_max,
)

# Print rounded results for data and seed
print("ADD and Thin density evaluation:")
print("================================")
print(
    f"{data_name} (Seed: {seed}): MMD: {mmd:.3f}, Wasserstein: {wasserstein:.3f}"
)

/home/cyang314/ucr_work/add-thin/add_thin/data.py:659: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loader = torch.load(path, map_location=device)
/home/cyang314/.conda/env